In [1]:
import math
import csv
import random


def loadcsv(filename):
    lines = csv.reader(open(filename, 'r'))
    dataset = list(lines)
    
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
        
    return dataset

In [2]:
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    
    return [trainSet, copy]

In [3]:
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if(vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

In [4]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))

def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers]) / float(len(numbers)-1)
    return math.sqrt(variance)
    
def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

In [5]:
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2) / (2*math.pow(stdev,2))))
    return (1/(math.sqrt(2*math.pi) * stdev)) * exponent

def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
            
    return bestLabel

def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        if testSet[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [8]:
def NaiveBayes():
    filename = '5.csv'
    splitRatio = 0.67
    dataset = loadcsv(filename)
    
    print("\nThe length of the Dataset:", len(dataset))
    
    print("\nThe Dataset Splitting into Training and Testing \n")
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    
    print("\nNumber of Rows in Training Set: {0} rows".format(len(trainingSet)))
    print("\nNumber of Rows in Testing Set: {0} rows".format(len(testSet)))
    
    print("\nFirst Five Rows of Training Set:\n")
    for i in range(0,5):
        print(trainingSet[i], "\n")
        
    print("\nFirst Five Rows of Testing Set:\n")
    for i in range(0,5):
        print(testSet[i], "\n")
        
    summaries = summarizeByClass(trainingSet)
    print("\nModel Summaries:\n", summaries)
    
    predictions = getPredictions(summaries, testSet)
    print("\nPredictions:\n", predictions)
    
    accuracy = getAccuracy(testSet, predictions)
    print("\nAccuracy: {0}%".format(accuracy))

In [9]:
NaiveBayes()


The length of the Dataset: 768

The Dataset Splitting into Training and Testing 


Number of Rows in Training Set: 514 rows

Number of Rows in Testing Set: 254 rows

First Five Rows of Training Set:

[3.0, 193.0, 70.0, 31.0, 0.0, 34.9, 0.241, 25.0, 1.0] 

[2.0, 84.0, 50.0, 23.0, 76.0, 30.4, 0.968, 21.0, 0.0] 

[0.0, 125.0, 96.0, 0.0, 0.0, 22.5, 0.262, 21.0, 0.0] 

[8.0, 118.0, 72.0, 19.0, 0.0, 23.1, 1.476, 46.0, 0.0] 

[8.0, 109.0, 76.0, 39.0, 114.0, 27.9, 0.64, 31.0, 1.0] 


First Five Rows of Testing Set:

[0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0, 1.0] 

[2.0, 197.0, 70.0, 45.0, 543.0, 30.5, 0.158, 53.0, 1.0] 

[4.0, 110.0, 92.0, 0.0, 0.0, 37.6, 0.191, 30.0, 0.0] 

[10.0, 139.0, 80.0, 0.0, 0.0, 27.1, 1.441, 57.0, 0.0] 

[5.0, 166.0, 72.0, 19.0, 175.0, 25.8, 0.587, 51.0, 1.0] 


Model Summaries:
 {1.0: [(4.935483870967742, 3.6385516917108616), (140.43010752688173, 33.0991623473538), (69.85483870967742, 21.344418935721215), (22.456989247311828, 17.993716007296413), (103.23655